<a href="https://colab.research.google.com/github/GuanRuLai/Python-Deep-Learning/blob/main/Pytorch_ANN(Multi_Label_Categorization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing

## Import library

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

import HappyML.preprocessor as pp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader

## Check if there is GPU to use

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU instead.")

There are 1 GPU(s) available.


## Import dataset & Split independent variables and dependent variable

In [ ]:
dataset = load_iris()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=["Iris_type"])
print(X.head())
print(y.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
   Iris_type
0          0
1          0
2          0
3          0
4          0


## Split training set and testing set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
137                6.4               3.1                5.5               1.8
84                 5.4               3.0                4.5               1.5
27                 5.2               3.5                1.5               0.2
127                6.1               3.0                4.9               1.8
132                6.4               2.8                5.6               2.2
..                 ...               ...                ...               ...
9                  4.9               3.1                1.5               0.1
103                6.3               2.9                5.6               1.8
67                 5.8               2.7                4.1               1.0
117                7.7               3.8                6.7               2.2
47                 4.6               3.2                1.4               0.2

[120 rows x 4 columns]
     sepal length (cm)  sepal width (cm)

## Feature scaling

In [ ]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
print(X_train)
print(X_test)

[[ 0.61303014  0.10850105  0.94751783  0.736072  ]
 [-0.56776627 -0.12400121  0.38491447  0.34752959]
 [-0.80392556  1.03851009 -1.30289562 -1.33615415]
 [ 0.25879121 -0.12400121  0.60995581  0.736072  ]
 [ 0.61303014 -0.58900572  1.00377816  1.25412853]
 [-0.80392556 -0.82150798  0.04735245  0.21801546]
 [-0.21352735  1.73601687 -1.19037495 -1.20664002]
 [ 0.14071157 -0.82150798  0.72247648  0.47704373]
 [ 0.02263193 -0.12400121  0.21613346  0.34752959]
 [-0.09544771 -1.05401024  0.10361279 -0.04101281]
 [ 1.0853487  -0.12400121  0.94751783  1.1246144 ]
 [-1.39432376  0.34100331 -1.41541629 -1.33615415]
 [ 1.20342834  0.10850105  0.72247648  1.38364267]
 [-1.04008484  1.03851009 -1.24663528 -0.81809761]
 [-0.56776627  1.50351461 -1.30289562 -1.33615415]
 [-1.04008484 -2.4490238  -0.1776889  -0.30004108]
 [ 0.73110978 -0.12400121  0.94751783  0.736072  ]
 [ 0.96726906  0.57350557  1.0600385   1.64267094]
 [ 0.14071157 -1.98401928  0.66621615  0.34752959]
 [ 0.96726906 -1.2865125   1.11

## Convert data into tensor

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
print(X_train_tensor)
print(X_test_tensor)
print(Y_train_tensor)
print(Y_test_tensor)

tensor([[ 0.6130,  0.1085,  0.9475,  0.7361],
        [-0.5678, -0.1240,  0.3849,  0.3475],
        [-0.8039,  1.0385, -1.3029, -1.3362],
        [ 0.2588, -0.1240,  0.6100,  0.7361],
        [ 0.6130, -0.5890,  1.0038,  1.2541],
        [-0.8039, -0.8215,  0.0474,  0.2180],
        [-0.2135,  1.7360, -1.1904, -1.2066],
        [ 0.1407, -0.8215,  0.7225,  0.4770],
        [ 0.0226, -0.1240,  0.2161,  0.3475],
        [-0.0954, -1.0540,  0.1036, -0.0410],
        [ 1.0853, -0.1240,  0.9475,  1.1246],
        [-1.3943,  0.3410, -1.4154, -1.3362],
        [ 1.2034,  0.1085,  0.7225,  1.3836],
        [-1.0401,  1.0385, -1.2466, -0.8181],
        [-0.5678,  1.5035, -1.3029, -1.3362],
        [-1.0401, -2.4490, -0.1777, -0.3000],
        [ 0.7311, -0.1240,  0.9475,  0.7361],
        [ 0.9673,  0.5735,  1.0600,  1.6427],
        [ 0.1407, -1.9840,  0.6662,  0.3475],
        [ 0.9673, -1.2865,  1.1163,  0.7361],
        [-0.3316, -1.2865,  0.0474, -0.1705],
        [ 2.1481, -0.1240,  1.2851

# Neural network processing

## Define model

In [ ]:
class IrisModel(nn.Module):

  # define frameworks of each neural layer
  def __init__(self):
    super(IrisModel, self).__init__()

    # define neural layers
    self.fc1 = nn.Linear(4, 16)
    self.fc2 = nn.Linear(16, 16)
    self.fc3 = nn.Linear(16, 3)

    # define weight initializers of each layer(default)
    init.xavier_normal_(self.fc1.weight)
    init.xavier_normal_(self.fc2.weight)
    init.xavier_normal_(self.fc3.weight)

  # define forward propagation function to connect layers(including activation function)
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

# initialize model
model = IrisModel().to(device)

# define loss function
criterion = nn.CrossEntropyLoss()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Model training & evaluation

In [ ]:
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset =  TensorDataset(X_test_tensor, Y_test_tensor)
# cut dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
epochs = 100
for epoch in range(epochs):
  model.train() # weights can be modified

  correct = 0
  total = 0

  for X_batch, Y_batch in train_loader:
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
    optimizer.zero_grad() # return zero of every previous batch

    Y_pred = model(X_batch)
    loss = criterion(Y_pred, Y_batch.squeeze())
    loss.backward() # calculate gradient(min loss weights)
    optimizer.step() # update weights

    _, predicted = torch.max(Y_pred.data, 1) # get the index of max value in each row of axis 1
    total += Y_batch.size(0) # get the number of samples
    correct += (predicted == Y_batch.squeeze()).sum().item()

  accuracy = correct / total
  print(f"Epoch: {epoch + 1}/{epochs}, Loss: {loss.item():.4f}, Acc: {accuracy:.4f}")

Epoch: 1/100, Loss: 1.1860, Acc: 0.1667
Epoch: 2/100, Loss: 1.1813, Acc: 0.2083
Epoch: 3/100, Loss: 1.1581, Acc: 0.2500
Epoch: 4/100, Loss: 1.0811, Acc: 0.2917
Epoch: 5/100, Loss: 1.0382, Acc: 0.3833
Epoch: 6/100, Loss: 1.0784, Acc: 0.4583
Epoch: 7/100, Loss: 1.0624, Acc: 0.4917
Epoch: 8/100, Loss: 1.0318, Acc: 0.5417
Epoch: 9/100, Loss: 0.9995, Acc: 0.5583
Epoch: 10/100, Loss: 0.9639, Acc: 0.5917
Epoch: 11/100, Loss: 0.9280, Acc: 0.6167
Epoch: 12/100, Loss: 1.0118, Acc: 0.6167
Epoch: 13/100, Loss: 0.9587, Acc: 0.6500
Epoch: 14/100, Loss: 0.9496, Acc: 0.6583
Epoch: 15/100, Loss: 0.9514, Acc: 0.6750
Epoch: 16/100, Loss: 0.9159, Acc: 0.6750
Epoch: 17/100, Loss: 0.8972, Acc: 0.6833
Epoch: 18/100, Loss: 0.8833, Acc: 0.6833
Epoch: 19/100, Loss: 0.8680, Acc: 0.6917
Epoch: 20/100, Loss: 0.8823, Acc: 0.6917
Epoch: 21/100, Loss: 0.9314, Acc: 0.7000
Epoch: 22/100, Loss: 0.9045, Acc: 0.7000
Epoch: 23/100, Loss: 0.6554, Acc: 0.7083
Epoch: 24/100, Loss: 0.7346, Acc: 0.7083
Epoch: 25/100, Loss: 0.72

## Test evaluation

In [ ]:
model.eval() # weights cannot be modified(frozen)

y_true = []
y_pred = []

with torch.no_grad(): # close the gradient calculation mechanism
  correct = 0
  total = 0

  for X_batch, Y_batch in test_loader:
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

    Y_pred = model(X_batch)
    _, predicted = torch.max(Y_pred.data, 1)
    total += Y_batch.size(0)
    correct += (predicted == Y_batch.squeeze()).sum().item()
    y_true.extend(Y_batch.squeeze().cpu().numpy())
    y_pred.extend(predicted.cpu().numpy())
  print(f"Test Acc: {correct / total:.4f}")

Test Acc: 1.0000


## Answer prediction

In [ ]:
results_df = pd.DataFrame({
    "Y_true": y_true,
    "Y_pred": y_pred
})
print(results_df)

    Y_true  Y_pred
0        2       2
1        1       1
2        0       0
3        2       2
4        0       0
5        2       2
6        0       0
7        1       1
8        1       1
9        1       1
10       2       2
11       1       1
12       1       1
13       1       1
14       1       1
15       0       0
16       1       1
17       1       1
18       0       0
19       0       0
20       2       2
21       1       1
22       0       0
23       0       0
24       2       2
25       0       0
26       0       0
27       1       1
28       1       1
29       0       0
